<h1>Содержание<span class="tocSkip"></span></h1>
<li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Подготовка-Без-Bert" data-toc-modified-id="Подготовка-Без-Bert-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Подготовка Без Bert</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Классификация токсичных комментариев (BERT)

Делаем инструмент, который позволит искать токсичные комментарии в описании товаров интернет магазина для дальнейшей модерации. 
Нужна модель со значением метрики качества `F1` >= 0.75

**Описание данных**

`text` - текст комментария  
`toxic` - целевой признак  

## Подготовка

In [1]:

import re 

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords

import torch

import warnings
warnings.filterwarnings('ignore')

import numpy as np

import pandas as pd
pd.set_option('display.max_colwidth', 200)

!pip install transformers
import transformers

from tqdm import tqdm
from tqdm import notebook

!pip install catboost
from catboost import CatBoostClassifier

from datetime import datetime

from lightgbm import LGBMClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df = pd.read_csv('/content/toxic_comments.csv')

In [3]:
df.head()

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remo...",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about...",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tid...",0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


Комментарии на английском языке.

In [4]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 80.1 MB


Пропуски отсутствуют.  
Объем занимаемой памяти небольшой, оптимизация не требуется.  

In [5]:
df['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

Есть дисбаланс классов.  
Попробуем обучить модели как есть, но учтем пропорции классов при дальнейшем разбиении.  

Обработаем текст с помощью `BERT`, а затем `TF-IDF`  
В конце сравним результаты.  

In [6]:

model_class, tokenizer_class, pretrained_weights = (
    transformers.BertModel,
    transformers.BertTokenizer,
    'bert-base-uncased'
)

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)


Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Мы загрузили натренированную модель `BERT`, и её токенайзер.  
Возьмем количество строк кратное 1 батчу обучения.  

In [7]:
text = df['text'].iloc[:159550].copy()

In [8]:

tokenized = text.apply(
    lambda x: tokenizer.encode(
        x.lower(),               # приводим к нижнему регистру
        add_special_tokens=True, # добавляем токены начала и конца строки
        max_length=512,          # ограничиваем максимальную длину последовательности токенов (ограничение BERT = 512)
        truncation=True          # обрезаем последовательность по длине
        )
    )


In [9]:
tokenized[0]

[101,
 7526,
 2339,
 1996,
 10086,
 2015,
 2081,
 2104,
 2026,
 5310,
 18442,
 13076,
 12392,
 2050,
 5470,
 2020,
 16407,
 1029,
 2027,
 4694,
 1005,
 1056,
 3158,
 9305,
 22556,
 1010,
 2074,
 8503,
 2006,
 2070,
 3806,
 2044,
 1045,
 5444,
 2012,
 2047,
 2259,
 14421,
 6904,
 2278,
 1012,
 1998,
 3531,
 2123,
 1005,
 1056,
 6366,
 1996,
 23561,
 2013,
 1996,
 2831,
 3931,
 2144,
 1045,
 1005,
 1049,
 3394,
 2085,
 1012,
 6486,
 1012,
 16327,
 1012,
 4229,
 1012,
 2676,
 102]

Токенизатор выполнил разбиение строк на слова, добавил спец. символы в начало и конец строки, подставил токены вместо слов.  

In [10]:
padded = np.array([i + [0]*(512-len(i)) for i in tokenized.values])

In [11]:
padded[0]

array([  101,  7526,  2339,  1996, 10086,  2015,  2081,  2104,  2026,
        5310, 18442, 13076, 12392,  2050,  5470,  2020, 16407,  1029,
        2027,  4694,  1005,  1056,  3158,  9305, 22556,  1010,  2074,
        8503,  2006,  2070,  3806,  2044,  1045,  5444,  2012,  2047,
        2259, 14421,  6904,  2278,  1012,  1998,  3531,  2123,  1005,
        1056,  6366,  1996, 23561,  2013,  1996,  2831,  3931,  2144,
        1045,  1005,  1049,  3394,  2085,  1012,  6486,  1012, 16327,
        1012,  4229,  1012,  2676,   102,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

Мы добавили нули ко всем более коротким чем 512 токенов строкам (в новом массиве).  
Это называется паддинг (набивка).  
Паддинг помогает `BERT` обрабатывать данные быстрее, теперь это один двумерный массив, а не списки токенов разной длины.

In [12]:
attention_mask = np.where(padded != 0, 1, 0)

In [13]:
attention_mask[0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Теперь мы создали 'маску внимания', она поможет `BERT` обращать внимание только на токены, игнорируя добавленные нули.  

In [14]:

%%time

batch_size = 50  # часть выборки для одной итерации цикла
embeddings = []

for i in notebook.tqdm(range(len(text) // batch_size)):

        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).cuda() # отправляем вычисления на GPU
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)]).cuda()

        with torch.no_grad():
            model.cuda()
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
            
        # берем из трехмерного тензора только признаки классов которые определила модель
        embeddings.append(batch_embeddings[0][:, 0, :].cpu().numpy()) 

features = np.concatenate(embeddings) # собираем признаки классов в один массив


  0%|          | 0/3191 [00:00<?, ?it/s]

CPU times: user 3h 1min 10s, sys: 18.5 s, total: 3h 1min 29s
Wall time: 3h 1min 3s


In [15]:
features[0]

array([ 2.50974655e-01,  5.05420975e-02, -2.41744774e-03, -2.32462451e-01,
       -3.06949586e-01, -3.72466475e-01,  5.56583345e-01,  5.22573292e-01,
        2.18740851e-01, -4.53634709e-01, -1.21842764e-01,  1.39128178e-01,
       -2.81780571e-01,  3.17809582e-01,  4.88872468e-01,  1.60018146e-01,
       -4.66759115e-01,  5.40292621e-01,  2.01923281e-01,  6.74564689e-02,
       -8.11745133e-03, -3.54632229e-01,  1.30774125e-01, -7.25800842e-02,
        8.74608383e-02, -7.07302690e-02, -1.83526278e-01, -3.89893353e-01,
       -2.17679322e-01,  6.55460218e-03, -2.71842003e-01,  2.25761712e-01,
        8.48642290e-02, -1.45496756e-01,  7.10862637e-01, -3.48647773e-01,
        1.80211157e-01, -7.64387697e-02,  3.92408520e-01,  2.65866071e-01,
       -8.13826919e-02,  1.14874534e-01,  4.10840213e-01,  1.30273417e-01,
        7.13752434e-02, -6.27828715e-03, -3.24370885e+00, -1.47058398e-01,
       -1.93766594e-01, -3.22880805e-01,  4.09788758e-01, -3.40376526e-01,
        2.70481229e-01,  

Мы взяли только нужную информацию.  
`BERT` проводит свою классификацию предложений, и ставит токен класса на первое место.  
Это готовые признаки для обучения и тестирования.  
Важное замечание: работая с батчами мы берем только то количество строк которое кратно размеру батча.  
Это означает что в нашем случеа мы обработали 159550 строк из 159571.  

In [16]:

X_train, X_test, y_train, y_test = train_test_split(
    features,
    df['toxic'].iloc[:159550],
    test_size=0.25,
    stratify=df['toxic'].iloc[:159550],  # сохраняем пропорции классов
    random_state=42
)


Разделили выборку, переходим к обучению.  

In [17]:

# для скорости проверки, диапазоны параметров уже указаны близко к оптимальным, вычисленным ранее

grid = {
    'CBC': {
        'model': CatBoostClassifier(),
        'parameters': {
            'max_depth': [x for x in range(6, 8, 2)],
            'n_estimators': [x for x in range(100, 125, 25)],
            'silent': [True] # не будем печатать каждую итерацию
        }
    },
    'LGBMC': {
        'model': LGBMClassifier(),
        'parameters': {
            'objective': ['binary'],
            'verbosity': [-1],
            'num_iterations': [750]
        }
    }
}


In [18]:

F1 = make_scorer(
    f1_score, # метрика по которой мы будем оценивать модель это F1 мера
    greater_is_better=True
)

scores = []
start_time = datetime.now()

for model, parameter in grid.items():
    GS_CV = GridSearchCV(
        parameter['model'], 
        parameter['parameters'], 
        scoring = F1, 
        cv=2
        )
    
    GS_CV.fit(
        X_train,
        y_train
        )
    
    scores.append({
    'model': model,
    'best_train_score': round(GS_CV.best_score_, 3), 
    'best_parameters': GS_CV.best_params_
    })
        
print('Время работы (ч:м:с.мс) {}'.format(datetime.now()-start_time))


Время работы (ч:м:с.мс) 0:26:55.025502


In [19]:

models_analysis = pd.DataFrame(scores, columns=[
    'model',
    'best_parameters',
    'best_train_score',
    'test_score',
    'test_training_time', 
    'test_prediction_time'
])
models_analysis.iloc[:, :3].sort_values(by=['best_train_score'], ascending=False)


,model,best_parameters,best_train_score
1,LGBMC,"{'num_iterations': 750, 'objective': 'binary', 'verbosity': -1}",0.688
0,CBC,"{'max_depth': 6, 'n_estimators': 100, 'silent': True}",0.661


Нам не удалось достичь значения метрики `F1`>=0.75 при использовании `BERT`  
Попробуем обойтись более простыми методами.  

### Подготовка Без BERT

In [20]:
corpus = df['text']

Теперь нам понадобится лемматизация и очистка от ненужных символов.  

In [21]:

def lemmatize(text):
    lem = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text.lower())
    lemm_text =  ' '.join([lem.lemmatize(w) for w in word_list])
    return lemm_text


def clear_text(text):
    sub_text = re.sub(r'[^a-zA-Z ]', ' ', text)
    return ' '.join(sub_text.split())


In [22]:
lemm_clear_corpus = corpus.apply(lambda x: lemmatize(clear_text(x)))

In [23]:

corpus_train, corpus_test, y_train, y_test = train_test_split(
    lemm_clear_corpus,
    df['toxic'],
    test_size=0.25,
    stratify=df['toxic'],
    random_state=12345
)


Определим слова которые не несут смысла, чтобы их удалить.  

In [24]:

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [25]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

Мы определили векторизатор.  
Сначала он будет работать в `Pipeline`, затем обработаем им тексты полностью.  

In [26]:
# чтобы Векторизатор не учился на валидационной части при кросс-валидации тренировочной выборки:
# воспользуемся Pipeline

grid = {
    'CBC': {
        'model': Pipeline([
            ('vekt', count_tf_idf),
            ('cbc', CatBoostClassifier())
            ]),
        'parameters': {
            'cbc__max_depth': [x for x in range(6, 8, 2)],
            'cbc__n_estimators': [x for x in range(100, 125, 25)],
            'cbc__silent': [True] 
        }
    },
    'LGBMC': {
        'model': Pipeline([
            ('vekt', count_tf_idf),
            ('lgbmc', LGBMClassifier())
            ]),
        'parameters': {
            'lgbmc__objective': ['binary'],
            'lgbmc__verbosity': [-1],
            'lgbmc__num_iterations': [750]
        }
    }
}


In [27]:

F1 = make_scorer(
    f1_score,
    greater_is_better=True
)

scores = []
start_time = datetime.now()

for model, parameter in grid.items():
    GS_CV = GridSearchCV(
        parameter['model'], 
        parameter['parameters'], 
        scoring = F1, 
        cv=2
        )
    
    GS_CV.fit(
        corpus_train,
        y_train
        )
    
    scores.append({
    'model': model,
    'best_train_score': round(GS_CV.best_score_, 3), 
    'best_parameters': GS_CV.best_params_
    })
        
print('Время работы (ч:м:с.мс) {}'.format(datetime.now()-start_time))


Время работы (ч:м:с.мс) 0:27:39.198011


In [28]:

models_analysis = pd.DataFrame(scores, columns=[
    'model',
    'best_parameters',
    'best_train_score',
    'test_score',
    'test_training_time', 
    'test_prediction_time'
])
models_analysis.iloc[:, :3].sort_values(by=['best_train_score'], ascending=False)


,model,best_parameters,best_train_score
1,LGBMC,"{'lgbmc__num_iterations': 750, 'lgbmc__objective': 'binary', 'lgbmc__verbosity': -1}",0.765
0,CBC,"{'cbc__max_depth': 6, 'cbc__n_estimators': 100, 'cbc__silent': True}",0.734


 Лучшей моделью стал `LGBMClassifier`  
 Удалось превзойти значение `F1`>=0.75

## Обучение

Проверим модель на тестовой выборке.  

In [29]:

tf_idf_train = count_tf_idf.fit_transform(corpus_train)
tf_idf_test = count_tf_idf.transform(corpus_test)


Обработали тексты токенизатором.

In [30]:
model = LGBMClassifier(num_boost_round=750, objective='binary', verbosity=-1)

In [31]:
model.fit(tf_idf_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_boost_round=750, num_leaves=31,
               objective='binary', random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0, verbosity=-1)

In [32]:
predictions = model.predict(tf_idf_test)

In [33]:
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98     35837
           1       0.88      0.70      0.78      4056

    accuracy                           0.96     39893
   macro avg       0.92      0.84      0.88     39893
weighted avg       0.96      0.96      0.96     39893



Результат стал еще лучше.  
Сравним c `DummyClassifier`, константно предскажем везде единицу:

In [34]:

clf = DummyClassifier(strategy='constant', constant=1)

clf.fit(tf_idf_train, y_train)
predictions = clf.predict(tf_idf_test)

print('F1 score для константной модели: ', f1_score(y_test, predictions).round(2))


F1 score для константной модели:  0.18


Наша модель явно лучше.

## Выводы

 Для обработки текста мы попробовали сразу применить мощную модель `BERT`.  
 Однако для нашего случая, с ограниченными ресурсами, вполне можно обойтись и без нейронной сети.  
 Тем более, что с ней нам не удалось достичь желаемого результата.    
 С более простой моделью обработки тескта мы достигли `F1` меры > 0.75.  
 Лучшей моделью классификации стал `LGBMClassifier`  
 Не всегда применение ресурсных моделей оправдано на практике.  